In [18]:
from __future__ import annotations

import numpy as np
import pandas as pd
from pathlib import Path



In [19]:
BASE_DIR = Path("../ma-data/ma/enrollment/Extracted Data").resolve()
OUTPUT_DIR = Path("data/output").resolve()
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


In [20]:
def monthlist_for_year(y: int) -> list[str]:
    return [f"{m:02d}" for m in range(1, 13)]


CONTRACT_COLUMNS = [
    "contractid",
    "planid",
    "org_type",
    "plan_type",
    "partd",
    "snp",
    "eghp",
    "org_name",
    "org_marketing_name",
    "plan_name",
    "parent_org",
    "contract_date",
]

CONTRACT_DTYPES = {
    "contractid": "string",
    "planid": "float64",
    "org_type": "string",
    "plan_type": "string",
    "partd": "string",
    "snp": "string",
    "eghp": "string",
    "org_name": "string",
    "org_marketing_name": "string",
    "plan_name": "string",
    "parent_org": "string",
    "contract_date": "string",
}

In [21]:
def read_contract(path: Path) -> pd.DataFrame:
    return pd.read_csv(
        path,
        skiprows=1,
        header=None,
        names=CONTRACT_COLUMNS,
        dtype=CONTRACT_DTYPES,
        encoding="latin1",
        low_memory=False,
    )

In [22]:
def load_month(m: str, y: int) -> pd.DataFrame:
    c_path = BASE_DIR / f"CPSC_Contract_Info_{y}_{m}.csv"

    contract_info = (
        read_contract(c_path)
        .drop_duplicates(subset=["contractid", "planid"], keep="first")
    )

    return contract_info.assign(month=int(m), year=y)

In [23]:
def build_plan_year(y: int) -> pd.DataFrame:
    monthlist = monthlist_for_year(y)

    plan_year = pd.concat(
        [load_month(m, y) for m in monthlist],
        ignore_index=True,
    )

    plan_year = plan_year.sort_values(
        ["contractid", "planid", "month"],
        kind="mergesort",
    )

    return plan_year.reset_index(drop=True)

In [24]:
def collapse_to_yearly_panel(plan_year: pd.DataFrame) -> pd.DataFrame:
    return (
        plan_year
        .groupby(["contractid", "planid", "year"], dropna=False)
        .agg(
            org_type=("org_type", "last"),
            plan_type=("plan_type", "last"),
            partd=("partd", "last"),
            snp=("snp", "last"),
            eghp=("eghp", "last"),
            org_name=("org_name", "last"),
            org_marketing_name=("org_marketing_name", "last"),
            plan_name=("plan_name", "last"),
            parent_org=("parent_org", "last"),
            contract_date=("contract_date", "last"),
        )
        .reset_index()
    )


In [25]:
if __name__ == "__main__":
    YEAR = 2018

    plan_year = build_plan_year(YEAR)
    final_plans = collapse_to_yearly_panel(plan_year)

    final_plans.to_csv(
        OUTPUT_DIR / "ma_plan_year_2018.csv",
        index=False,
    )


In [26]:
  print("Done")

Done


### Answer 1

In [27]:
# Table 1: Count of plans by plan type (2018)

table1 = (
    final_plans
    .groupby("plan_type", dropna=False)
    .size()
    .reset_index(name="n_plans")
    .sort_values("n_plans", ascending=False)
)

table1


,plan_type,n_plans
3,HMO/HMOPOS,2678
6,Medicare Prescription Drug Plan,1011
4,Local PPO,966
8,National PACE,258
10,Regional PPO,109
0,1876 Cost,101
7,Medicare-Medicaid Plan HMO/HMOPOS,54
9,PFFS,50
2,HCPP - 1833 Cost,9
5,MSA,5


### Answer 2

In [28]:
filtered_q2 = final_plans[
    (final_plans["snp"] != "Yes") &
    (final_plans["eghp"] != "Yes") &
    (~final_plans["contractid"].str.startswith("800"))
]

In [29]:
table1_q2 = (
    filtered_q2
    .groupby("plan_type", dropna=False)
    .size()
    .reset_index(name="n_plans")
    .sort_values("n_plans", ascending=False)
)

table1_q2

,plan_type,n_plans
2,HMO/HMOPOS,1569
5,Medicare Prescription Drug Plan,794
3,Local PPO,569
7,National PACE,258
0,1876 Cost,93
6,Medicare-Medicaid Plan HMO/HMOPOS,54
9,Regional PPO,49
8,PFFS,46
1,HCPP - 1833 Cost,9
4,MSA,3


### Answer 3

In [30]:
ENROLL_DIR = Path("../ma-data/ma/enrollment/Extracted Data").resolve()
SA_DIR     = Path("../ma-data/ma/service-area/Extracted Data").resolve()

YEAR = 2018
MONTHS = [f"{m:02d}" for m in range(1, 13)]


In [39]:
plan_year = build_plan_year(2018)
print(plan_year.columns)


Index(['contractid', 'planid', 'org_type', 'plan_type', 'partd', 'snp', 'eghp',
       'org_name', 'org_marketing_name', 'plan_name', 'parent_org',
       'contract_date', 'month', 'year'],
      dtype='object')


In [40]:
def build_plan_year_2018_enrollment() -> pd.DataFrame:
    months = [f"{m:02d}" for m in range(1, 13)]
    out = []

    for m in months:
        e_path = ENROLL_DIR / f"CPSC_Enrollment_Info_2018_{m}.csv"
        df = pd.read_csv(
            e_path,
            skiprows=1,
            header=None,
            names=["contractid","planid","ssa","fips","state","county","enrollment"],
            encoding="latin1",
            na_values=["*"],
            low_memory=False,
            dtype={
                "contractid":"string",
                "planid":"float64",
                "state":"string",
                "county":"string",
                "enrollment":"float64",
            }
        )
        df["month"] = int(m)
        df["year"] = 2018
        out.append(df)

    return pd.concat(out, ignore_index=True)



In [41]:
plan_year = build_plan_year_2018_enrollment()

In [42]:
final_plans = (
    plan_year
    .groupby(["contractid","planid","state","county","year"], dropna=False)["enrollment"]
    .mean()
    .reset_index(name="avg_enrollment")
)

In [43]:
plans_with_sa = final_plans.merge(
    approved_cc,
    on=["contractid","state","county"],
    how="left"
)

plans_with_sa["approved"] = plans_with_sa["approved"].fillna(0).astype(int)


In [45]:
plans_q3.columns


Index(['contractid', 'planid', 'state', 'county', 'year', 'avg_enrollment',
       'approved'],
      dtype='object')

In [46]:
contract_2018 = pd.read_csv(
    "../ma-data/ma/enrollment/Extracted Data/CPSC_Contract_Info_2018_12.csv",
    skiprows=1,
    header=None,
    names=[
        "contractid","planid","org_type","plan_type","partd","snp","eghp",
        "org_name","org_marketing_name","plan_name","parent_org","contract_date"
    ],
    encoding="latin1",
    low_memory=False
)

contract_2018 = contract_2018.drop_duplicates(subset=["contractid","planid"])[
    ["contractid","planid","plan_type","snp","eghp"]
]


In [47]:
plans_with_sa = plans_with_sa.merge(
    contract_2018,
    on=["contractid","planid"],
    how="left"
)


In [48]:
plans_q3 = plans_with_sa[
    (plans_with_sa["approved"] == 1) &
    (plans_with_sa["snp"] != "Yes") &
    (plans_with_sa["eghp"] != "Yes") &
    (~plans_with_sa["contractid"].str.startswith("800"))
].copy()


In [49]:
q3_table = (
    plans_q3
    .groupby("plan_type")["avg_enrollment"]
    .mean()
    .reset_index(name="avg_enrollment")
)

q3_table


,plan_type,avg_enrollment


#### Answer 3: After restricting the sample to approved counties and excluding SNP plans, EGHP plans, and 800-series contracts, there are no remaining Medicare Advantage plans in 2018. As a result, average enrollment by plan type cannot be computed because no plans satisfy all required criteria simultaneously.